## CMB example

Plan:
* we make a simulation of the CMB, a gaussian random field
  * Each Fourier mode is an independent Gaussian random number with amplitude given by $\sqrt{C_l}$ where $C_l$ is the power spectrum.
* we take a cutout -- the map is no longer periodic (power leakage)
* we taper / window the map to cut down on the power leakage

In [ ]:
%pylab inline

* define a function that gives the FFT frequency in 2d.

In [ ]:
def dist(N):
    return float(N) * sqrt(outer(fftfreq(N)**2, 1) + outer(1,fftfreq(N)**2))

* we will make a 2d grid of gaussian noise, uncorrelated in pixel space

In [ ]:
N = 500 #let's make a 500x500-pixel map
width_sr = 10 * (pi / 180.) #100-degree by 100-degree map

uncorrelated = random.randn(N,N)  #white/uncorrelated Gaussian noise 


* let's make an image

In [ ]:
imshow(uncorrelated, cmap = 'bwr')
colorbar()
title('uncorrelated')

In [ ]:
fftUncorrelated = fft.fft2(uncorrelated)

* The Fourier transform of white noise is white noise

In [ ]:
imshow(fft.fftshift(abs(fftUncorrelated)), cmap = 'bwr')

* Fundamental mode: $2\pi/X$ where $X$ is the width of the map.

In [ ]:
dl = 2* pi / (width_sr)

In [ ]:
dl

* A routine to put the CMB power spectrum onto the 2d grid

In [ ]:
import scipy.interpolate

def getGridPower(filenameCAMB, N, dl):
    power1d = numpy.loadtxt( filenameCAMB)
    myF = scipy.interpolate.interp1d(power1d[:,0], \
                                     power1d[:,1] / power1d[:,0]**2, bounds_error = False)
    gridPower = nan_to_num(myF(dl * dist(N)))
    gridPower[0,0]
    return gridPower


In [ ]:
#gridpower = gridPower1d.reshape(N,N)
#imshow(gridPower)
#gridPower.shape
#myF(dl * sqrt(outer(fftfreq(N)**2, fftfreq(N)**2)) + dl)

In [ ]:
gridPower = getGridPower('../lecture3/fiducial_scalCls.dat', N, dl)


* An image of the 2d power spectrum

In [ ]:
imshow(fftshift(log10(gridPower + 1e-10))) 
colorbar()


We multiply the FFT of the white noise by the square root of the power spectrum

In [ ]:

myCMBfft = sqrt(gridPower) * fftUncorrelated

* The fourier transform of our CMB map

In [ ]:
imshow(fftshift(abs(log10(myCMBfft + 0))))

* A slice through the FFT of the CMB, along the $x$ axis.

In [ ]:
semilogy(abs(myCMBfft[:,0]))

The map in real space:

In [ ]:
myCMBMap = real(nan_to_num(fft.ifft2(sqrt(gridPower) * fftUncorrelated)))

In [ ]:
imshow(myCMBMap, cmap = 'bwr')

* This can be compared with the WMAP or Planck data.

* Now let's take a cutout of this image.  We'll take a 200x200 cutout.

In [ ]:
Ncutout = 200
cutoutMap = myCMBMap[100:100+Ncutout, 100:100+Ncutout]

In [ ]:
imshow(cutoutMap, cmap = 'bwr')

In [ ]:
cutoutFFT = fft.fft2(cutoutMap)

In [ ]:
semilogy(abs(cutoutFFT[0,:]))

* The map isn't periodic anymore and there is a lot of "leakage" of power from large to small scales.
* Let's define a window function to reduce the amount of leakage.

In [ ]:
window = outer(ones(Ncutout) , 1+cos(linspace(0,2*pi,Ncutout) + pi)) * \
    outer(1+cos(linspace(0,2*pi,Ncutout)+pi), ones(Ncutout))
window /= mean(window)

In [ ]:
imshow(window, cmap = 'Greys')
colorbar()

* Now the window times the map.

In [ ]:
imshow(window * cutoutMap, cmap = 'bwr')

In [ ]:
cutoutWindowedFFT = fft.fft2(window * cutoutMap ) 

* Now we plot the power (along the $x$ axis) for the unwindowed map; windowed map; and theory (note we haven't been careful about overall normalizaiton; we are paying more attention to the shape of the spectrum).

In [ ]:
figure(figsize=(8,6))
nmax = Ncutout/2
semilogy( abs(cutoutFFT[0,:]), label = 'cutout')
semilogy( abs(cutoutWindowedFFT[0,:] ), label = 'cutout and windowed')
gridPowerCutout = getGridPower('../lecture3/fiducial_scalCls.dat', Ncutout, dl)
gridPowerCutout[0,0] = nan
rescale = 400. #we haven't been careful about the overall amplitude
gridPowerCutout[gridPowerCutout*rescale < 2e-3] = nan #cut out modes that aren't measured for comparison

semilogy(abs(gridPowerCutout[0,:]) * rescale, label = 'theory')
legend()
xlim([0,Ncutout])